# Building a Project with ChatGPT API to auto generate professional tweets

In [86]:
import os
import openai #needed for error handling
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

### Authenticate to the API using the API Key
#### Pull from environment variables or use api_key = ("your_key_here") to hardcode the key

In [87]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    

In [88]:
# Each interaction with the AI assistant is a new session so the entire chat/message history, 
# including user prompts and assistant responses must be included in each exchange with the
# model/assistant so that it "remembers" the entire session
context = []
def collect_messages(role, message): #keeps track of the message exchange between user and assistant
    context.append({'role': role, 'content':f"{message}"})

In [89]:
# Sends the prompts to the model for a completion/response
def get_completion(temperature=0): 
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context, 
            temperature=temperature, 
        )

        print("\n Assistant: ", response.choices[0].message.content, "\n")

        return response.choices[0].message.content
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [90]:
# We may need this code in the next project to generate a full interaction with the user about the content
# Start the conversation between the user and the AI assistant/chatbot
"""
while True:
    summary = get_completion()
    
    collect_messages('assistant', summary) #stores the response from the AI assistant
    
    user_prompt = input('User: ') #input box for entering prompt
    
    if user_prompt == 'exit': #end the conversation with the AI assistant
        print("\n Goodbye")
        break

    collect_messages('user', user_prompt) #stores the user prompt
""" 

'\nwhile True:\n    summary = get_completion()\n    \n    collect_messages(\'assistant\', summary) #stores the response from the AI assistant\n    \n    user_prompt = input(\'User: \') #input box for entering prompt\n    \n    if user_prompt == \'exit\': #end the conversation with the AI assistant\n        print("\n Goodbye")\n        break\n\n    collect_messages(\'user\', user_prompt) #stores the user prompt\n'

In [104]:

context.clear()
collect_messages('user', 'Pick a random startup website in the crypto, artificial intelligence or quantom computing domains. You should only give the link to the website as an answer, without the prefix  https://www. ,Do not repeat yourself') #stores the user prompt
print(context)
random_site = get_completion()
print(random_site)

[{'role': 'user', 'content': 'Pick a random startup website in the crypto, artificial intelligence or quantom computing domains. You should only give the link to the website as an answer, without the prefix  https://www. ,Do not repeat yourself'}]

 Assistant:  coinbase.com 

coinbase.com


In [106]:
#sets the persona for the AI assistant using a system message
context.clear()
collect_messages('system', """You are a friendly AI assistant that 
                            helps compose professional-sounding tweets 
                            for Twitter that often go viral based on a 
                            website I provide. You will generate a summary 
                            of the website in up to 218 characters only!""")
#add the url as the user prompt, and ask to generate a summary
collect_messages('user', random_site)
summary = get_completion()


 Assistant:  Coinbase is a leading cryptocurrency exchange platform that allows users to buy, sell, and store various digital currencies securely. #crypto #trading 



In [107]:
# Create images based on the website summary
def generate_image(summary):
    print(summary)
    
    try:
        response = client.images.generate(
          model="dall-e-3",
          prompt=summary,
          size="1024x1024",
          quality="standard",
          n=1, #select the number of images you want generated
        )
        
        image_url = response.data[0].url #URLs will expire after an hour, so we also need to save the file locally 

        return image_url
    except openai.APIError as e:
        print(e.http_status)
        print(e.error)
        return e.error

In [108]:
url = generate_image(summary)

Coinbase is a leading cryptocurrency exchange platform that allows users to buy, sell, and store various digital currencies securely. #crypto #trading


In [109]:
#display the image inline
from IPython.display import Image
Image(url=url)

In [12]:
#the twitter api lib
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [110]:
import tweepy
import requests

# Authenticate to Twitter API
consumer_key = os.getenv("API_KEY")
consumer_secret = os.getenv("API_SECRET_KEY")
access_token = os.getenv("ACCESS_TOKEN")   
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET_KEY") 

#download image to notebook
def download_image(imageURL):
    print("downloading - ", imageURL)
    
    img_data = requests.get(imageURL).content
    with open('dalle_image.jpg', 'wb') as handler:
        handler.write(img_data)
    
    return "dalle_image.jpg"

#upload image media using V1 of Twitter API
def upload_image(image):
    auth = tweepy.OAuth1UserHandler(
       consumer_key,
       consumer_secret,
       access_token,
       access_token_secret
    )

    api = tweepy.API(auth)
    media = api.media_upload(filename=image) 
    
    return media

#send the tweet using V2 of the Twitter API
def send_tweet(summary, image):
    client = tweepy.Client(
        consumer_key=consumer_key, consumer_secret=consumer_secret,
        access_token=access_token, access_token_secret=access_token_secret
    )

    #upload image to Twitter servers and get the media metadata
    media = upload_image(image)
    media_ids = [media.media_id]
    
    #send the tweet
    print(media_ids)
    print(summary)
    response = client.create_tweet(text=summary, media_ids=media_ids)
    
    print(f"https://twitter.com/user/status/{response.data['id']}")

In [111]:
#download image to the notebook
image_name = download_image(url)

downloading -  https://oaidalleapiprodscus.blob.core.windows.net/private/org-Xpq7QD5G5VDB9ibrYpVDO9at/user-XXLYjR7rCcqu0vxfhe5jO30C/img-YBwDUzcTjSxlYBOnC2HbMMmp.png?st=2024-09-10T08%3A39%3A47Z&se=2024-09-10T10%3A39%3A47Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-09-10T00%3A00%3A44Z&ske=2024-09-11T00%3A00%3A44Z&sks=b&skv=2024-08-04&sig=FSt3OMC8u6sc3B1LrJ%2BLGiY/DY/Z/2hzyWRbg05rpIc%3D


In [112]:
#post new tweet
send_tweet(summary, image_name)

[1833440580807868416]
Coinbase is a leading cryptocurrency exchange platform that allows users to buy, sell, and store various digital currencies securely. #crypto #trading
https://twitter.com/user/status/1833440585299988966


In [ ]:
# This code checks the Twitter API limits on our account
import tweepy

# Your Twitter API credentials
consumer_key = os.getenv("API_KEY")
consumer_secret = os.getenv("API_SECRET_KEY")
access_token = os.getenv("ACCESS_TOKEN")   
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET_KEY") 


# Authenticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


# Create API object
api = tweepy.API(auth)

# Get rate limit status
rate_limit_status = api.rate_limit_status()

# Print rate limits for specific endpoints
'''print("Rate limits:")
for endpoint, limit in rate_limit_status['resources'].items():
    for method, status in limit.items():
        print(f"{endpoint} - {method}:")
        print(f"  Limit: {status['limit']}")
        print(f"  Remaining: {status['remaining']}")
        print(f"  Reset time: {status['reset']}")
        print()'''

# Check specific endpoint (e.g., search tweets)
search_tweets_limit = rate_limit_status['resources']['tweets&POST']['/tweets&POST']
print("Search Tweets Endpoint:")
print(f"  Limit: {search_tweets_limit['limit']}")
print(f"  Remaining: {search_tweets_limit['remaining']}")
print(f"  Reset time: {search_tweets_limit['reset']}")
print(rate_limit_status)
